# Emergency Vehicle Detection Model Training

This notebook covers the training of YOLOv8 model for emergency vehicle detection.

In [9]:
import os
from pathlib import Path
from ultralytics import YOLO
import torch
import yaml
import shutil
from sklearn.model_selection import train_test_split
import cv2
import numpy as np
from tqdm import tqdm

## 1. Data Preparation

In [8]:
# Setup directories
DATASET_ROOT = Path('../Dataset')
OUTPUT_DIR = DATASET_ROOT / 'prepared'

# Create directories
for split in ['train', 'val', 'test']:
    (OUTPUT_DIR / split / 'images').mkdir(parents=True, exist_ok=True)
    (OUTPUT_DIR / split / 'labels').mkdir(parents=True, exist_ok=True)

# Load dataset config
with open(DATASET_ROOT / 'dataset.yaml', 'r') as f:
    dataset_config = yaml.safe_load(f)

# Class mapping
class_names = {
    'Ambulance': 0,
    'Fire Engine': 1,
    'Police': 2,
    'Non Emergency': 3
}

## 2. Preprocessing Functions

In [9]:
def preprocess_image(img_path, target_size=(640, 640)):
    """Preprocess image for YOLOv8"""
    img = cv2.imread(str(img_path))
    if img is None:
        return None
        
    # Calculate padding
    h, w = img.shape[:2]
    ratio = min(target_size[0]/w, target_size[1]/h)
    new_w, new_h = int(w * ratio), int(h * ratio)
    
    # Resize
    img = cv2.resize(img, (new_w, new_h))
    
    # Pad
    pad_w = (target_size[0] - new_w) // 2
    pad_h = (target_size[1] - new_h) // 2
    padded = np.full((target_size[1], target_size[0], 3), 114, dtype=np.uint8)
    padded[pad_h:pad_h+new_h, pad_w:pad_w+new_w] = img
    
    return padded

def create_label(img_path, class_id):
    """Create YOLO format label"""
    img = cv2.imread(str(img_path))
    h, w = img.shape[:2]
    
    # Convert to YOLO format (class x_center y_center width height)
    return f"{class_id} 0.5 0.5 0.8 0.8\n"  # Approximate bounding box

## 3. Process and Split Dataset

In [ ]:
def process_dataset():
    """Process and split dataset"""
    all_images = []
    
    # Collect all images
    for class_name, class_id in class_names.items():
        class_path = DATASET_ROOT / class_name
        if not class_path.exists():
            continue
            
        for img_path in class_path.glob('*.jpg'):
            all_images.append((img_path, class_id))
    
    # Split dataset
    train_data, test_val = train_test_split(all_images, test_size=0.3, random_state=42)
    val_data, test_data = train_test_split(test_val, test_size=0.5, random_state=42)
    
    # Process splits
    for split_name, split_data in [
        ('train', train_data),
        ('val', val_data),
        ('test', test_data)
    ]:
        for img_path, class_id in tqdm(split_data, desc=f'Processing {split_name}'):
            # Preprocess image
            processed = preprocess_image(img_path)
            if processed is None:
                continue
                
            # Save image
            out_img_path = OUTPUT_DIR / split_name / 'images' / img_path.name
            cv2.imwrite(str(out_img_path), processed)
            
            # Create and save label
            label = create_label(img_path, class_id)
            out_label_path = OUTPUT_DIR / split_name / 'labels' / f"{img_path.stem}.txt"
            with open(out_label_path, 'w') as f:
                f.write(label)

# Process dataset
process_dataset()

## 4. Update Dataset Configuration

In [11]:
# Update paths in dataset config
dataset_config['path'] = str(OUTPUT_DIR)
dataset_config['train'] = str(OUTPUT_DIR / 'train')
dataset_config['val'] = str(OUTPUT_DIR / 'val')
dataset_config['test'] = str(OUTPUT_DIR / 'test')

# Save updated config
with open(OUTPUT_DIR / 'dataset.yaml', 'w') as f:
    yaml.dump(dataset_config, f)

## 5. Model Training

In [ ]:
# Create a new dataset config
dataset_yaml = {
    'path': str(OUTPUT_DIR.absolute()),  # dataset root dir
    'train': str((OUTPUT_DIR / 'train' / 'images').absolute()),  # train images
    'val': str((OUTPUT_DIR / 'val' / 'images').absolute()),  # val images
    'test': str((OUTPUT_DIR / 'test' / 'images').absolute()),  # test images
    'names': {v: k for k, v in class_names.items()},  # class names
    'nc': len(class_names)  # number of classes
}

# Save the updated dataset config
yaml_path = OUTPUT_DIR / 'dataset.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(dataset_yaml, f, sort_keys=False)

# Load base model
model = YOLO('yolov8n.pt')

# Training arguments
training_args = {
    'data': str(yaml_path.absolute()),  # use absolute path
    'epochs': 100,
    'imgsz': 640,
    'batch': 16,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'workers': 8,
    'name': 'emergency_vehicles',
    'patience': 50,
    'save_period': 10,
    'pretrained': True,
    'lr0': 0.001,
    'lrf': 0.01,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3.0,
    'warmup_momentum': 0.8,
    'warmup_bias_lr': 0.1,
    'box': 7.5,
    'cls': 0.5,
    'hsv_h': 0.015,
    'hsv_s': 0.7,
    'hsv_v': 0.4,
    'translate': 0.1,
    'scale': 0.5,
    'fliplr': 0.5,
    'mosaic': 1.0
}

# Start training
results = model.train(**training_args)

## 6. Training Analysis

In [ ]:
import matplotlib.pyplot as plt

# Check if model exists and has results
if 'model' in globals() and hasattr(model, 'results'):
	metrics = model.results  # Access results from the trained model
	for key in metrics.keys():
		plt.figure(figsize=(10, 6))
		plt.plot(metrics[key])
		plt.title(f'Training Metrics - {key}')
		plt.xlabel('Epoch')
		plt.ylabel(key)
		plt.grid(True)
		plt.show()
else:
	print("Model or training results not found. Make sure the model training cell was executed successfully.")

## 7. Save Model

In [8]:
from pathlib import Path
import shutil

# Save the best model
best_model_path = Path('./runs/detect/emergency_vehicles/weights/best.pt')
if best_model_path.exists():
    output_path = Path('./yolov8n.pt')
    shutil.copy(best_model_path, output_path)
    print(f"Best model saved to: {output_path}")
else:
    print("Best model file not found. Make sure training was completed successfully.")

Best model file not found. Make sure training was completed successfully.
